In [3]:
import os, glob, time
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
os.makedirs(os.path.join("Real Property Dataset"),exist_ok = True)
Dataset_Path = os.path.join("Real Property Dataset", "Real Property Information.csv")

In [5]:
def Scrape_Data(driver): 
    House_Information = []
    Page = driver.page_source
    Source = BeautifulSoup(Page, "lxml")

    Titles = Source.find_all("span", class_="pr-title js__card-title")
    Areas = Source.find_all("span", class_="re__card-config-area js__card-config-item")
    Bedrooms = Source.find_all("span", class_="re__card-config-bedroom js__card-config-item")
    Bathrooms = Source.find_all("span", class_="re__card-config-toilet js__card-config-item")
    Location = Source.find_all("div", class_="re__card-location")
    Prices = Source.find_all("span", class_="re__card-config-price js__card-config-item")
    
    for title, area, bedroom, bathroom, price, local in zip(Titles, Areas, Bedrooms, Bathrooms, Prices, Location):
        House_Information.append({
            'House Title': title.text.strip(),
            'Area': area.text.strip(),
            'Bedrooms': bedroom.find('span').text.strip(),
            'Bathrooms': bathroom.find('span').text.strip(),
            'Location': local.text.strip(),
            'Price': price.text.strip()
        })
    
    return House_Information

In [3]:
def Browser_Automation():
    driver = webdriver.Edge()
    driver.set_window_size(1280, 720)
    URL = "https://batdongsan.com.vn/ban-nha-rieng-tp-hcm?cIds=325,163"
    driver.get(URL)
    
    House_List = []
    
    def close_popups():
        while True:
            try:
                close_button = WebDriverWait(driver, 0.5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "span.close")))
                close_button.click()
                time.sleep(0.25)
            except Exception:
                break
            
    while True:
        try:
            close_popups() 
            House_Data = Scrape_Data(driver)
            House_List.extend(House_Data)
            Next_Page = WebDriverWait(driver, 1.5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.re__pagination-icon i.re__icon-chevron-right--sm")))
            Next_Page.click()
            time.sleep(1)                                   
        except Exception:
            break
            
    driver.quit()
    return House_List

Scrape_House_List = pd.DataFrame(Browser_Automation())
Scrape_House_List.to_csv(Dataset_Path, header=True, index=False, encoding="utf-8-sig")

In [23]:
def Clean_Area_Column(Dataset):
    try:
        Dataset["Area"].astype(float)
        return Dataset["Area"].astype(float)
    except:
        Dataset["Area"] = Dataset["Area"].str.replace("m²", "", regex=False).str.replace(".", "")
        Dataset["Area"] = Dataset["Area"].str.replace(",", ".").str.strip()
        return Dataset["Area"].astype(float)

def Clean_Locations_Column(Dataset):
    Location_List = [
        "Quận 1", "Quận 2", "Quận 3", "Quận 4", "Quận 5", "Quận 6", "Quận 7", "Quận 8", "Quận 9",
        "Quận 10", "Quận 11", "Quận 12", "Bình Thạnh", "Gò Vấp", "Phú Nhuận", "Tân Bình",
        "Tân Phú", "Bình Tân", "Thủ Đức", "Nhà Bè", "Hóc Môn", "Bình Chánh", "Củ Chi", "Cần Giờ" 
    ]
    Dataset["Location"] = Dataset["Location"].astype(str)
    Dataset["Location"] = Dataset["Location"].str.replace("·\r\n", "")
    
    def Clean_Locations_Data(Location):
        Parts = Location.split(",")
        for Part in Parts:
            Part = Part.strip()
            if Part in Location_List:
                return Part
    Dataset["Location"] = Dataset["Location"].apply(Clean_Locations_Data)
    return Dataset["Location"]

def Clean_Price_Column(Dataset):
    try:
        Dataset["Price"].astype(float)
        return Dataset["Price"].astype(float)
    except:
        Dataset["Price"] = Dataset["Price"].str.replace("Giá thỏa thuận", "0")
        Dataset["Price"] = Dataset["Price"].str.replace(".", "")
        Dataset["Price"] = Dataset["Price"].str.replace(",", ".")
        def Clean_Price_Data(Price):
            Parts = Price.split()
            try:
                if "tỷ" in Parts:
                    return float(Parts[0])
                elif "triệu" in Parts:
                    return float(Parts[0]) / 1000
            except ValueError:
                return 0

        Dataset["Price"] = Dataset["Price"].apply(Clean_Price_Data)
        return Dataset["Price"].astype(float)

In [25]:
def Data_Cleaning():
    House_List = pd.read_csv(Dataset_Path)
    House_List["Area"] = Clean_Area_Column(House_List)
    House_List["Location"] = Clean_Locations_Column(House_List)
    House_List["Price"] = Clean_Price_Column(House_List)
    
    House_List.to_csv(Dataset_Path, header=True, index=False, encoding="utf-8-sig")
    
Data_Cleaning()
House_List = pd.read_csv(Dataset_Path)

In [26]:
def House_Price_Statistics():
    if 'House Information.db' not in glob.glob("House Information.db"):
        House_List = pd.read_csv(os.path.join("Real Property Dataset", "Real Property Information.csv"))
        Connection = sqlite3.connect('House Information.db')
        House_List.to_sql('House Information', Connection, if_exists='replace', index=False)
        print("Database already created!")
        return Connection
    else: return sqlite3.connect('House Information.db')

def Exc(Query):
    Connection = House_Price_Statistics()
    return pd.read_sql_query(Query, Connection)

In [27]:
Connection = House_Price_Statistics()
Cursor = Connection.cursor ()
Data = pd.read_sql_query("SELECT * FROM 'House Information'", Connection)
Data

Database already created!


,House Title,Area,Bedrooms,Bathrooms,Location,Price
0,Biệt thự 2 mặt tiền Nguyễn Đổng Chi: Cuộc sống...,302.00,5,5,Quận 7,NaN
1,Lên sóng căn siêu phẩm ngay chợ HTT đường Lê V...,24.00,2,2,Gò Vấp,3.20
2,Siêu phẩm đẹp say đắm trệt lửng 2 lầu 3PN có g...,50.00,3,4,Gò Vấp,6.99
3,"Bán nhà phố 2 mặt view, ngay QL 13, Thủ Đức, g...",68.00,4,6,Quận 3,6.40
4,Dinh thự ven sông biệt lập The Mizuki DT từ 53...,647.00,5,7,Thủ Đức,63.00
...,...,...,...,...,...,...
27498,"Bán nhà ở tại đường Lý Thánh Tông, Tân Thới Hò...",88.95,1,2,Tân Phú,6.20
27499,Ngân hàng thanh lý nhà phố phường Tân Chánh Hi...,127.30,3,3,Quận 1,5.40
27500,"Ngân hàng thanh lý nhà phố đường Bùi Quang Là,...",104.90,2,1,Quận 10,7.30
27501,"Ngân hàng thanh lý nhà 4 tầng tại Quang Trung,...",66.30,4,5,Quận 7,5.60


In [28]:
Query = """
select Location, count("House Title") as "Number of Houses", avg(Price) as "Average Price", avg("Area") as "Average Area"
from 'House Information'
where Location is not null
group BY Location;
"""
Exc(Query)

,Location,Number of Houses,Average Price,Average Area
0,Bình Chánh,215,25.283152,139.959721
1,Bình Thạnh,3623,29.369511,198.333265
2,Bình Tân,1461,27.239943,142.412348
3,Cần Giờ,2,4.000000,77.500000
4,Củ Chi,334,28.102359,180.314731
5,Gò Vấp,1941,26.719113,133.768867
6,Hóc Môn,91,10.449551,128.137473
7,Nhà Bè,1080,29.315733,131.699019
8,Phú Nhuận,869,32.352374,583.341415
9,Quận 1,942,55.243752,144.739862


In [29]:
Urban_Areas = [
    "Quận 1", "Quận 2","Quận 3", "Quận 4", "Quận 5", "Quận 6", "Quận 7", "Quận 8", "Quận 9", "Quận 10", 
    "Quận 11", "Quận 12", "Bình Thạnh", "Gò Vấp", "Phú Nhuận", "Tân Bình", "Tân Phú", "Bình Tân", "Thủ Đức"
]

Urban_Areas_House_List = House_List[House_List["Location"].isin(Urban_Areas)]
Rural_Areas_House_List = House_List[~House_List["Location"].isin(Urban_Areas)]

### Khu vực nông thôn



In [30]:
Stats = Rural_Areas_House_List['Price'].describe()
pd.DataFrame(Stats)

,Price
count,1553.000000
mean,27.792070
std,68.127407
min,0.690000
25%,6.300000
50%,11.000000
75%,25.000000
max,1150.000000
